In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
#import seaborn as sns
from datetime import datetime
#import scipy.stats as stats
#import statsmodels.api as sm
import re
from shapely.geometry import Point
import geopandas as gpd
#import folium
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import n_colors
import chart_studio
import chart_studio.plotly as py
import plotly.figure_factory as ff

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [2]:
airbnb = pd.read_csv('../data/airbnb_df.csv')

In [3]:
airbnb.head()

,listing_id,comments_concatenated,name,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,listing_url,description,neighborhood_overview,host_since,host_listings_count,property_type,accommodates,bathrooms_text,bedrooms,beds,amenities,minimum_nights_avg_ntm,maximum_nights_avg_ntm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,zip_code,GEOID,house_price_2021-01-31,house_pct_change,rentals_2021-01-31,rental_price_pct_change,new_restaurants,available_beer,str_permits_2020,str_permits_growth,crimes,total_pop_2010,total_pop_2019,total_pop_change,total_pop_pct_change,pop_over25_2010,pop_over25_2019,pop_over25_change,pop_over25_pcg_change,total_households_2010,total_households_2019,total_households_change,total_households_pct_change,white_pct_2010,white_pct_2019,white_value_change,white_pct_change,bach_pct_2010,bach_pct_2019,bach_value_change,bach_pct_change,rent_pct_2010,rent_pct_2019,rent_value_change,renter_pct_change,median_hhi_2010,median_hhi_2019,median_hhi_value_change,median_hhi_pct_change,poverty_pct_2010,poverty_pct_2019,poverty_value_change,poverty_pct_change
0,6422,I can't say enough about how wonderful it was ...,Nashville Charm,12172,36.17315,-86.73581,40,30,674,4.69,1,267,https://www.airbnb.com/rooms/6422,30 day or more rental during COVID. Show COVID...,Historic East Nashville is home to many new an...,2009-04-03,0.0,Private room in house,2,1 private bath,2.0,3.0,"[""Hair dryer"", ""Bathtub"", ""Lock on bedroom doo...",30.0,365.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,37206.0,4.703701e+10,412476.0,38.31,NaN,NaN,1.0,2.0,114.0,114.0,1165.0,2544.0,2100.0,-444.0,-0.174528,1703.0,1639.0,-64.0,-0.037581,1140.0,926.0,-214.0,-0.187719,0.657626,0.940952,0.283327,0.430833,0.408691,0.585723,0.177032,0.433170,0.320175,0.240821,-0.079355,-0.247848,46000.0,91643.0,45643.0,0.992239,10.6,10.2,-0.4,-0.037736
1,25613,Nell and Michael made my stay pleasant and eas...,Room in Historic East Nashville Craftsman Home,95811,36.17826,-86.74162,75,2,510,3.87,1,63,https://www.airbnb.com/rooms/25613,Serve up some sweet sounds on the piano and wi...,This home is in a walkable neighborhood with i...,2010-03-18,1.0,Private room in house,2,1 private bath,1.0,1.0,"[""Hair dryer"", ""Carbon monoxide alarm"", ""Air c...",2.0,1125.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,37206.0,4.703701e+10,412476.0,38.31,NaN,NaN,3.0,12.0,101.0,101.0,1048.0,2485.0,2525.0,40.0,0.016097,1895.0,1938.0,43.0,0.022691,1234.0,1092.0,-142.0,-0.115073,0.889738,0.930297,0.040559,0.045585,0.472296,0.742002,0.269707,0.571055,0.416532,0.274725,-0.141806,-0.340446,52372.0,116800.0,64428.0,1.230199,21.6,4.1,-17.5,-0.810185
2,136015,"Very relaxing, beautiful property, and warm / ...",Apart. on the Ridge near Opryland,666322,36.19464,-86.67195,103,30,50,0.43,2,190,https://www.airbnb.com/rooms/136015,Fantastic View! Overlooks farms and is on the ...,Dead end street with great neighbors!,2011-06-05,4.0,Entire apartment,2,1 bath,1.0,1.0,"[""Fire extinguisher"", ""Wifi"", ""Dryer"", ""Washer...",30.0,1125.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,37214.0,4.703702e+10,278031.0,65.43,1581.0,22.18,9.0,112.0,48.0,48.0,4429.0,5411.0,6024.0,613.0,0.113288,4451.0,4970.0,519.0,0.116603,2566.0,2714.0,148.0,0.057677,0.781186,0.760458,-0.020728,-0.026534,0.365311,0.430986,0.065675,0.179778,0.120811,0.179071,0.058261,0.482250,53590.0,68532.0,14942.0,0.278821,3.6,6.9,3.3,0.916667
3,258817,Diana is such a gracious hostess. She has a b...,ButterflyRoom-private bath- open Jan 26,22296,36.16249,-86.58988,27,30,96,0.86,7,364,https://www.airbnb.com/rooms/258817,Morningstar House! Extended stay - 3+ months- ...,"I'm very near the Percy Priest Lake, natural ...",2009-06-19,16.0,Private room in house,2,1 private bath,1.0,2.0,"[""Hair dryer"", ""Air conditioning"", ""Smoke alar...",30.0,365.0,95.0,10.0,9.0,10.0,10.0,9.0,9.0,37076.0,4

In [4]:
#Replace inf in poverty_pct_change with max of column
m = airbnb.loc[airbnb['poverty_pct_change'] != np.inf, 'poverty_pct_change'].max()
airbnb['poverty_pct_change'].replace(np.inf,m,inplace=True)

In [5]:
# Keep only features that I will use to identify gentrification
gentrification_features = ['GEOID', 'house_pct_change', 'rental_price_pct_change', 'white_pct_change', 
                           'bach_pct_change', 'renter_pct_change', 'median_hhi_pct_change', 'poverty_pct_change']

In [6]:
gentrification_df = airbnb[gentrification_features].drop_duplicates('GEOID')

In [7]:
gentrification_df = gentrification_df[gentrification_df['GEOID'].notna()]

In [8]:
gentrification_df.head()

,GEOID,house_pct_change,rental_price_pct_change,white_pct_change,bach_pct_change,renter_pct_change,median_hhi_pct_change,poverty_pct_change
0,4.703701e+10,38.31,NaN,0.430833,0.433170,-0.247848,0.992239,-0.037736
1,4.703701e+10,38.31,NaN,0.045585,0.571055,-0.340446,1.230199,-0.810185
2,4.703702e+10,65.43,22.18,-0.026534,0.179778,0.482250,0.278821,0.916667
3,4.703702e+10,51.53,NaN,-0.176354,-0.037855,-0.028681,0.122428,1.088235
7,4.703702e+10,38.31,NaN,0.049556,0.387603,0.112385,0.720376,-0.228758


In [9]:
gentrification_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 4825
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   GEOID                    150 non-null    float64
 1   house_pct_change         149 non-null    float64
 2   rental_price_pct_change  87 non-null     float64
 3   white_pct_change         149 non-null    float64
 4   bach_pct_change          149 non-null    float64
 5   renter_pct_change        148 non-null    float64
 6   median_hhi_pct_change    147 non-null    float64
 7   poverty_pct_change       148 non-null    float64
dtypes: float64(8)
memory usage: 10.5 KB


In [10]:
gentrification_df.shape

(150, 8)

In [11]:
# Create list of features that need NaNs filled by medians
medians = ['house_pct_change', 'rental_price_pct_change', 'white_pct_change', 'bach_pct_change', 'renter_pct_change', 
           'median_hhi_pct_change', 'poverty_pct_change']

In [12]:
gentrification_df[medians] = gentrification_df[medians].fillna(gentrification_df.median())

In [13]:
gentrification_df.head()

,GEOID,house_pct_change,rental_price_pct_change,white_pct_change,bach_pct_change,renter_pct_change,median_hhi_pct_change,poverty_pct_change
0,4.703701e+10,38.31,18.35,0.430833,0.433170,-0.247848,0.992239,-0.037736
1,4.703701e+10,38.31,18.35,0.045585,0.571055,-0.340446,1.230199,-0.810185
2,4.703702e+10,65.43,22.18,-0.026534,0.179778,0.482250,0.278821,0.916667
3,4.703702e+10,51.53,18.35,-0.176354,-0.037855,-0.028681,0.122428,1.088235
7,4.703702e+10,38.31,18.35,0.049556,0.387603,0.112385,0.720376,-0.228758


In [14]:
gentrification_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 4825
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   GEOID                    150 non-null    float64
 1   house_pct_change         150 non-null    float64
 2   rental_price_pct_change  150 non-null    float64
 3   white_pct_change         150 non-null    float64
 4   bach_pct_change          150 non-null    float64
 5   renter_pct_change        150 non-null    float64
 6   median_hhi_pct_change    150 non-null    float64
 7   poverty_pct_change       150 non-null    float64
dtypes: float64(8)
memory usage: 10.5 KB


In [15]:
gentrification_df.describe()

,GEOID,house_pct_change,rental_price_pct_change,white_pct_change,bach_pct_change,renter_pct_change,median_hhi_pct_change,poverty_pct_change
count,1.500000e+02,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,4.703702e+10,53.128067,18.353067,0.774029,0.444372,0.210394,0.413937,1.135201
std,7.880766e+04,13.602770,4.888480,5.382851,0.742882,0.515318,0.386433,2.287961
min,4.703701e+10,31.110000,7.730000,-0.413595,-0.377566,-0.369817,-0.180354,-0.863874
25%,4.703701e+10,40.190000,18.350000,-0.091308,0.031430,-0.039293,0.163609,-0.183005
50%,4.703702e+10,51.350000,18.350000,-0.002333,0.184014,0.080751,0.341001,0.224314
75%,4.703702e+10,65.417500,18.350000,0.151780,0.588447,0.251190,0.581724,1.078472
max,4.703798e+10,74.310000,29.300000,64.071146,4.396047,2.792779,2.454128,7.461538


In [16]:
gentrification_df['GEOID'] = gentrification_df['GEOID'].astype(int).astype(str)

In [17]:
gentrification_df = gentrification_df.set_index('GEOID')

In [18]:
gentrification_df.head()

,house_pct_change,rental_price_pct_change,white_pct_change,bach_pct_change,renter_pct_change,median_hhi_pct_change,poverty_pct_change
GEOID,,,,,,,
47037012200,38.31,18.35,0.430833,0.433170,-0.247848,0.992239,-0.037736
47037012100,38.31,18.35,0.045585,0.571055,-0.340446,1.230199,-0.810185
47037015300,65.43,22.18,-0.026534,0.179778,0.482250,0.278821,0.916667
47037015610,51.53,18.35,-0.176354,-0.037855,-0.028681,0.122428,1.088235
47037019200,38.31,18.35,0.049556,0.387603,0.112385,0.720376,-0.228758


## Read in shapefile, Filter to Nashville

In [19]:
tracts = gpd.read_file('../data/cb_2018_47_tract_500k.shp')

In [20]:
tracts.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,47,093,001900,1400000US47093001900,47093001900,19,CT,2203514,13116,"POLYGON ((-83.90062 35.99202, -83.88800 36.000..."
1,47,093,003902,1400000US47093003902,47093003902,39.02,CT,3451146,0,"POLYGON ((-83.98563 36.00119, -83.98152 36.004..."
2,47,093,004404,1400000US47093004404,47093004404,44.04,CT,5964584,11451,"POLYGON ((-84.03695 35.93007, -84.02293 35.933..."
3,47,093,005708,1400000US47093005708,47093005708,57.08,CT,3557676,38601,"POLYGON ((-84.11800 35.86332, -84.11387 35.866..."
4,47,093,006103,1400000US47093006103,47093006103,61.03,CT,6437142,0,"POLYGON ((-84.07012 36.02888, -84.06820 36.031..."


In [21]:
tracts['FIPS'] = tracts.STATEFP + tracts.COUNTYFP

In [22]:
tracts = tracts[tracts['FIPS']=='47037'][['GEOID', 'geometry']]

### Create Interactive Visualization to Examine census tract by each feature's percent change
#### Not in Presentation, because widget does not publish with visual

In [53]:
@interact(feature = gentrification_df.columns.to_list())

def gentrification_map(feature) :
    percents_map = tracts.merge(gentrification_df.reset_index()[['GEOID', feature]], on='GEOID', how='left')
    percents_map[feature] = percents_map[feature].fillna(percents_map[feature].median())
    
    test_map = px.choropleth(percents_map, 
                        geojson=percents_map.geometry,
                        locations=percents_map.index, 
                        color=feature,
                        color_continuous_scale="Viridis"#,
                        #range_color=(0, 12),
                        #labels={'unemp':'unemployment rate'}
                        )
    test_map.update_geos(fitbounds="locations", visible=False)
    
    test_map.show()
    
    #test_map.write_html("/Users/mattparker/Documents/nss_projects/airbnb-sentiments/notebooks/test_map.html")
    #py.plot(test_map, filename = 'test_map', auto_open=True)

interactive(children=(Dropdown(description='feature', options=('house_pct_change', 'rental_price_pct_change', …

## Determine Gentrification by Ranking

In [24]:
gentrification_df.head()

,house_pct_change,rental_price_pct_change,white_pct_change,bach_pct_change,renter_pct_change,median_hhi_pct_change,poverty_pct_change
GEOID,,,,,,,
47037012200,38.31,18.35,0.430833,0.433170,-0.247848,0.992239,-0.037736
47037012100,38.31,18.35,0.045585,0.571055,-0.340446,1.230199,-0.810185
47037015300,65.43,22.18,-0.026534,0.179778,0.482250,0.278821,0.916667
47037015610,51.53,18.35,-0.176354,-0.037855,-0.028681,0.122428,1.088235
47037019200,38.31,18.35,0.049556,0.387603,0.112385,0.720376,-0.228758


In [25]:
gentrification_ranks = gentrification_df.copy()

In [26]:
gentrification_ranks['house_pct_change_rank'] = gentrification_ranks['house_pct_change'].rank(ascending=False, method='min').astype(int)
gentrification_ranks['rental_price_pct_change_rank'] = gentrification_ranks['rental_price_pct_change'].rank(ascending=False, method='min').astype(int)
gentrification_ranks['white_pct_change_rank'] = gentrification_ranks['white_pct_change'].rank(ascending=False, method='min').astype(int)
gentrification_ranks['bach_pct_change_rank'] = gentrification_ranks['bach_pct_change'].rank(ascending=False, method='min').astype(int)
gentrification_ranks['renter_pct_change_rank'] = gentrification_ranks['renter_pct_change'].rank(ascending=True, method='min').astype(int)
gentrification_ranks['median_hhi_pct_change_rank'] = gentrification_ranks['median_hhi_pct_change'].rank(ascending=False, method='min').astype(int)
gentrification_ranks['poverty_pct_change_rank'] = gentrification_ranks['poverty_pct_change'].rank(ascending=True, method='min').astype(int)

In [27]:
gentrification_ranks.head()

,house_pct_change,rental_price_pct_change,white_pct_change,bach_pct_change,renter_pct_change,median_hhi_pct_change,poverty_pct_change,house_pct_change_rank,rental_price_pct_change_rank,white_pct_change_rank,bach_pct_change_rank,renter_pct_change_rank,median_hhi_pct_change_rank,poverty_pct_change_rank
GEOID,,,,,,,,,,,,,,
47037012200,38.31,18.35,0.430833,0.433170,-0.247848,0.992239,-0.037736,118,37,20,47,8,12,57
47037012100,38.31,18.35,0.045585,0.571055,-0.340446,1.230199,-0.810185,118,37,62,39,2,7,2
47037015300,65.43,22.18,-0.026534,0.179778,0.482250,0.278821,0.916667,31,29,93,77,132,84,107
47037015610,51.53,18.35,-0.176354,-0.037855,-0.028681,0.122428,1.088235,65,37,135,128,41,122,113
47037019200,38.31,18.35,0.049556,0.387603,0.112385,0.720376,-0.228758,118,37,61,52,87,25,36


In [28]:
gentrification_ranks.columns

Index(['house_pct_change', 'rental_price_pct_change', 'white_pct_change', 'bach_pct_change', 'renter_pct_change', 'median_hhi_pct_change', 'poverty_pct_change', 'house_pct_change_rank', 'rental_price_pct_change_rank', 'white_pct_change_rank', 'bach_pct_change_rank', 'renter_pct_change_rank', 'median_hhi_pct_change_rank', 'poverty_pct_change_rank'], dtype='object')

In [29]:
gentrification_rank_counts = gentrification_ranks[['house_pct_change_rank', 'rental_price_pct_change_rank', 
                                                   'white_pct_change_rank', 'bach_pct_change_rank', 
                                                   'renter_pct_change_rank', 'median_hhi_pct_change_rank', 
                                                   'poverty_pct_change_rank']]

In [30]:
gentrification_rank_counts['total'] = gentrification_rank_counts.sum(axis=1)

In [31]:
gentrification_rank_counts['total_rank'] = gentrification_rank_counts['total'].rank(
    ascending=True, method='min').astype(int)

In [32]:
gentrification_rank_counts

,house_pct_change_rank,rental_price_pct_change_rank,white_pct_change_rank,bach_pct_change_rank,renter_pct_change_rank,median_hhi_pct_change_rank,poverty_pct_change_rank,total,total_rank
GEOID,,,,,,,,,
47037012200,118,37,20,47,8,12,57,299,20
47037012100,118,37,62,39,2,7,2,267,14
47037015300,31,29,93,77,132,84,107,553,93
47037015610,65,37,135,128,41,122,113,641,120
47037019200,118,37,61,52,87,25,36,416,41
...,...,...,...,...,...,...,...,...,...
47037018905,73,123,75,56,61,42,34,464,59
47037015629,50,37,112,40,68,99,108,514,74
47037010904,1,1,38,9,55,150,73,327,27


In [33]:
# Select the top 10% of census tracts
gentrification_rank_table = gentrification_rank_counts.drop(
    columns='total').sort_values(
    'total_rank', ascending=True).head(15)

In [34]:
gentrification_rank_table = gentrification_rank_table.reset_index()
gentrification_rank_table

,GEOID,house_pct_change_rank,rental_price_pct_change_rank,white_pct_change_rank,bach_pct_change_rank,renter_pct_change_rank,median_hhi_pct_change_rank,poverty_pct_change_rank,total_rank
0,47037012600,1,1,12,3,3,4,9,1
1,47037011800,1,1,10,16,4,5,13,2
2,47037011300,1,1,16,12,37,13,28,3
3,47037019400,17,37,13,18,67,1,42,4
4,47037013800,17,37,7,15,121,6,7,5
5,47037013700,17,37,9,10,59,71,17,6
6,47037013300,99,18,33,8,11,14,52,7
7,47037016000,1,37,4,6,136,2,53,8
8,47037016100,126,37,30,7,19,3,23,9
9,47037013202,99,18,69,1,34,29,4,10


In [35]:
gentrification_rank_table['GEOID'] = gentrification_rank_table['GEOID'].str[-5:]

In [36]:
gentrification_rank_table['neighborhood'] = ['McFerrin Park',
                                            'Cleveland Park',
                                            'Talbots Corner',
                                            'Germantown/Hope Gardens',
                                            'Osage/North Fisk',
                                            'MetroCenter/Buena Vista Heights',
                                            'The Nations',
                                            'Chestnut Hill',
                                            'Wedgewood-Houston',
                                            'Charlotte Park',
                                            'Woodbine',
                                            'South Inglewood',
                                            'Woodbine/Glencliff',
                                            'Lockeland Springs',
                                            'North Inglewood']#,
#                                             'Charlotte Ave',
#                                             'Glengarry/Glencliff Estates',
#                                             'Dickerson Pike/OHB',
#                                             'Whispering Hills',
#                                             'Shelby Park',
#                                             'Tennessee State University',
#                                              'Eastwood'
#                                             ]

In [37]:
gentrification_rank_table = gentrification_rank_table[['GEOID', 'neighborhood', 'house_pct_change_rank', 
                                                       'rental_price_pct_change_rank', 'white_pct_change_rank', 
                                                       'bach_pct_change_rank', 'renter_pct_change_rank', 
                                                       'median_hhi_pct_change_rank', 'poverty_pct_change_rank', 
                                                       'total_rank']]
gentrification_rank_table = gentrification_rank_table.rename(columns={'GEOID':'census_tract'})

In [38]:
gentrification_rank_table['census_tract'] = gentrification_rank_table['census_tract'] + ": " + gentrification_rank_table['neighborhood']

In [39]:
gentrification_rank_table = gentrification_rank_table.drop(columns='neighborhood')

In [40]:
gentrification_rank_table = gentrification_rank_table.rename(
    columns = {'census_tract': 'Census Tract: Neighborhood', 
              'house_pct_change_rank': 'Homes', 
              'rental_price_pct_change_rank': 'Rent',         
              'white_pct_change_rank': 'Race',          
              'bach_pct_change_rank': 'Educ.', 
              'renter_pct_change_rank': 'Tenure', 
              'median_hhi_pct_change_rank': 'HHI', 
              'poverty_pct_change_rank': 'Poverty', 
              'total_rank': 'Total'})

In [41]:
gentrification_rank_table

,Census Tract: Neighborhood,Homes,Rent,Race,Educ.,Tenure,HHI,Poverty,Total
0,12600: McFerrin Park,1,1,12,3,3,4,9,1
1,11800: Cleveland Park,1,1,10,16,4,5,13,2
2,11300: Talbots Corner,1,1,16,12,37,13,28,3
3,19400: Germantown/Hope Gardens,17,37,13,18,67,1,42,4
4,13800: Osage/North Fisk,17,37,7,15,121,6,7,5
5,13700: MetroCenter/Buena Vista Heights,17,37,9,10,59,71,17,6
6,13300: The Nations,99,18,33,8,11,14,52,7
7,16000: Chestnut Hill,1,37,4,6,136,2,53,8
8,16100: Wedgewood-Houston,126,37,30,7,19,3,23,9
9,13202: Charlotte Park,99,18,69,1,34,29,4,10


In [62]:
colors = n_colors('rgb(176, 5, 5)', 'rgb(13, 54, 158)', 150, colortype='rgb')
df = gentrification_rank_table

# fill_color = []
# n = len(df)
# for col in df:
#     if col=='GEOID':
#         [np.array('rgb(200, 15, 15)')[df[col].to_numpy()] for col in df]
#     else:
#         [np.array(colors)[df[col].to_numpy()] for col in df]
        #fill_color.append(df["color"].to_list()) ## Original fromm stackoverflow

fig = go.Figure(data=[go.Table(
    columnwidth = [120,25,25,25,25,25,25,25,25],
    header=dict(values=['<b>' + x + '</b>' for x in list(df.columns)],
                fill_color='gray',
                line_color='darkslategray',
                align='center',
                font=dict(color='black', family="Lato", size=16),
                height=30
                ),
    cells=dict(values=[df[column] for column in df.columns],
               fill_color = [np.array(colors)[df[column].to_numpy()] 
                             if column != 'Census Tract: Neighborhood' 
                             else np.array(['lightgray'] * 15) 
                             for column in df.columns],
               line_color='black',
               align=['left', 'center'],
               font=dict(color='black', family="Lato", size=16),
               height=30
               ))
])

# fig.update_layout(
# title="<b>Gentrification Feature Rank by Census Tract</b>",
# font=dict(
#     family="Lato",
#     size=18,
#     color="#000000"))

fig.show()

# fig.write_html("/Users/mattparker/Documents/nss_projects/airbnb-sentiments/plotly_vis/feature_table.html")
# py.plot(fig, filename = 'feature_table', auto_open=True)

'https://plotly.com/~matttparker/18/'

In [43]:
tracts_rankings = tracts.merge(gentrification_rank_counts.drop(columns='total').reset_index(), on='GEOID', how='left')

In [44]:
#tracts_rankings.head()

In [45]:
tracts_rankings['total_rank'] = tracts_rankings['total_rank'].fillna(tracts_rankings['total_rank'].max())

In [64]:
tracts_rankings['GEOID'] = tracts_rankings['GEOID'].str[-5:]

In [65]:
tracts_rankings.head()

,GEOID,geometry,house_pct_change_rank,rental_price_pct_change_rank,white_pct_change_rank,bach_pct_change_rank,renter_pct_change_rank,median_hhi_pct_change_rank,poverty_pct_change_rank,total_rank
0,15625,"POLYGON ((-86.65754 36.11937, -86.65558 36.122...",31.0,29.0,71.0,141.0,127.0,110.0,99.0,104.0
1,16000,"POLYGON ((-86.77265 36.14987, -86.76948 36.151...",1.0,37.0,4.0,6.0,136.0,2.0,53.0,8.0
2,17300,"POLYGON ((-86.75091 36.12039, -86.75051 36.121...",1.0,37.0,21.0,2.0,104.0,43.0,48.0,11.0
3,18409,"POLYGON ((-86.98225 36.07529, -86.98171 36.075...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0
4,19106,"POLYGON ((-86.74414 36.04342, -86.74375 36.047...",73.0,123.0,113.0,70.0,83.0,116.0,120.0,138.0


In [70]:
#percents_map[feature] = percents_map[feature].fillna(percents_map[feature].median())
colors = n_colors('rgb(176, 5, 5)', 'rgb(13, 54, 158)', 150, colortype='rgb')

ranking_map = px.choropleth(tracts_rankings, 
                    geojson=tracts_rankings.geometry,
                    locations=tracts_rankings.index, 
                    color='total_rank',
                    color_continuous_scale=colors,
                      labels={'GEOID': 'Census Tract',
                        'total_rank': 'Overall Rank',
                        'house_pct_change_rank':'Change in House Price',
                        'rental_price_pct_change_rank': 'Change in Rental House Price',
                        'white_pct_change_rank': 'Change in Percent White',
                        'bach_pct_change_rank': 'Change in Percent with Bachelor Degree or Higher',
                        'renter_pct_change_rank': 'Change in Percent Renting vs Owning Homes',
                        'median_hhi_pct_change_rank': 'Change in Median Household Income',
                        'poverty_pct_change_rank': 'Change in Percent under Poverty Level'},
                    hover_data=[#'index':False,
                                'GEOID',
                                'total_rank',
                                'house_pct_change_rank',
                               'rental_price_pct_change_rank',
                               'white_pct_change_rank',
                               'bach_pct_change_rank',
                               'renter_pct_change_rank',
                               'median_hhi_pct_change_rank',
                               'poverty_pct_change_rank']
                    )
ranking_map.update_geos(fitbounds="locations", visible=False)

ranking_map.show()

# ranking_map.write_html("/Users/mattparker/Documents/nss_projects/airbnb-sentiments/plotly_vis/ranking_map.html")
# py.plot(ranking_map, filename = 'ranking_map', auto_open=True)

'https://plotly.com/~matttparker/20/'

@interact(num = range(5, 26))

def gentrification_ranking_map(num) :
    
    changing_tracts_rankings = tracts_rankings.sort_values('total', ascending=False)['GEOID'].head(num).to_list()
    tracts_rankings_vis = tracts_rankings.copy()
    tracts_rankings_vis['gentrifying'] = np.where(tracts_rankings_vis.GEOID.isin(changing_tracts_rankings), True, False)
    
    fig = px.choropleth(tracts_rankings_vis, 
                        geojson=tracts_rankings_vis.geometry,
                        locations=tracts_rankings_vis.index, 
                        color='gentrifying',
                        color_continuous_scale="Viridis"#,
                        #range_color=(0, 12),
                        #labels={'unemp':'unemployment rate'}
                        )
    fig.update_geos(fitbounds="locations", visible=False)
    
    fig.show()

In [ ]:
tracts_rankings.head()

In [ ]:
changing_tracts_top = tracts_rankings.sort_values('total_rank', ascending=True)['GEOID'].head(15).to_list()
changing_tracts_top

In [ ]:
tracts_rankings['gentrifying'] = np.where(tracts_rankings.GEOID.isin(changing_tracts_top), True, False)

In [ ]:
tracts_rankings.head()

In [ ]:
leg_kwds = {'title': 'Changing Neighborhoods', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1.03), 'ncol': 1}

tracts_rankings.plot(column = 'gentrifying',
            figsize=(10, 10),
            edgecolor = 'black',
            cmap = 'Set2');

In [ ]:
tracts_rankings

In [ ]:
#tracts_rankings.to_file('../data/tracts_rankings.shp', index=False)

## Add Gentrifying boolean column to AirBNB Dataset

In [ ]:
airbnb_gentrification = airbnb.copy()

In [ ]:
airbnb_gentrification['gentrifying'] = np.where(
    airbnb_gentrification.GEOID.isin(changing_tracts_top), True, False)

In [ ]:
airbnb_gentrification.head()

In [ ]:
#airbnb_gentrification.to_csv('../data/airbnb_gentrification.csv')

## Determine gentrification by Quantiles (deprecated)

In [ ]:
gentrification_bools = gentrification_df.copy()

In [ ]:
gentrification_bools['house_pct_change_bool'] = np.where(
    gentrification_bools['house_pct_change'] > gentrification_bools['house_pct_change'].quantile(0.80), True, False)

gentrification_bools['rental_price_pct_change_bool'] = np.where(
    gentrification_bools['rental_price_pct_change'] > gentrification_bools['rental_price_pct_change'].quantile(0.80), True, False)

gentrification_bools['white_pct_change_bool'] = np.where(
    gentrification_bools['white_pct_change'] > gentrification_bools['white_pct_change'].quantile(0.80), True, False)

gentrification_bools['bach_pct_change_bool'] = np.where(
    gentrification_bools['bach_pct_change'] > gentrification_bools['bach_pct_change'].quantile(0.80), True, False)

gentrification_bools['renter_pct_change_bool'] = np.where(
    gentrification_bools['renter_pct_change'] < gentrification_bools['renter_pct_change'].quantile(0.20), True, False)

gentrification_bools['median_hhi_pct_change_bool'] = np.where(
    gentrification_bools['median_hhi_pct_change'] > gentrification_bools['median_hhi_pct_change'].quantile(0.80), True, False)

gentrification_bools['poverty_pct_change_bool'] = np.where(
    gentrification_bools['poverty_pct_change'] < gentrification_bools['poverty_pct_change'].quantile(0.20), True, False)

In [ ]:
gentrification_bools.head()

In [ ]:
gentrification_bools.columns

In [ ]:
gentrification_counts = gentrification_bools[['house_pct_change_bool', 'rental_price_pct_change_bool', 
                                           'white_pct_change_bool', 'bach_pct_change_bool', 
                                           'renter_pct_change_bool', 'median_hhi_pct_change_bool', 
                                           'poverty_pct_change_bool']]

In [ ]:
gentrification_counts['total'] = gentrification_counts.sum(axis=1)

## Visualize tracts by quantiles (deprecated)

In [ ]:
#1: Total Claims vs ODs, colored by percent of claims by selected opioid (all graphs use same color choice)
# Problem - Does not show significant results. 

@interact(feature = gentrification_counts.columns.to_list()[-9:-1])

def gentrification_map(feature) :
    tracts_visual = tracts.copy()
    
    visual_tracts = gentrification_counts[gentrification_counts[feature]==True].index.to_list()
    
    tracts_visual['visual'] = np.where(tracts.GEOID.isin(visual_tracts), True, False)
    
   # leg_kwds = {'title': 'Changing Neighborhoods', 'loc': 'upper left', 
   #         'bbox_to_anchor': (1, 1.03), 'ncol': 1}

    tracts_visual.plot(column = 'visual',
            figsize=(10, 10),
            edgecolor = 'black',
            cmap = 'Set2');

In [ ]:
gentrification_counts.head()

In [ ]:
tracts_booleans = tracts.merge(gentrification_counts.reset_index()[['GEOID', 'total']], on='GEOID', how='left')

In [ ]:
tracts_booleans['total'] = tracts_booleans['total'].fillna(0)

In [ ]:
tracts_booleans.head()

In [ ]:
tracts_booleans.plot(column = 'total',
            figsize=(10, 8),
            legend=True,
            edgecolor = 'black');